# [Workshop] Textual Knowledge Processing

<img src="ws_img_001.png">

# 0. Package Installation (one time job)

In [1]:
# !pip install -U spacy

In [2]:
# !python -m spacy download en_core_web_sm

In [3]:
# !pip install -U nltk

In [4]:
# import nltk
# nltk.download() # Select and Download the "popular" from "Collections"

# 1. Import Library

In [12]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import spacy

# 2. Text Preprocessing

## 2.1 Lower casing

In [13]:
def lower_casing(sentence):
    # Quiz: How to implement this function without using str.lower()?
    new_sentence = sentence.lower()
    return new_sentence

## 2.2 Abbreviation expansion

In [14]:
def expand_abbriviation(sentence):
    replacement_patterns = [
        (r'won\'t', 'will not'),
        (r'can\'t', 'cannot'),
        (r'i\'m', 'i am'),
        (r'ain\'t', 'is not'),
        (r'(\w+)\'ll', '\g<1> will'),
        (r'(\w+)n\'t', '\g<1> not'),
        (r'(\w+)\'ve', '\g<1> have'),
        (r'(\w+)\'s', '\g<1> is'),
        (r'(\w+)\'re', '\g<1> are'),
        (r'(\w+)\'d', '\g<1> would')]
    patterns = [(re.compile(regex), repl) for (regex, repl) in replacement_patterns]

    new_sentence = sentence
    for (pattern, repl) in patterns:
        (new_sentence, count) = re.subn(pattern, repl, new_sentence)
    return new_sentence

## 2.3 Punctuation removal

In [15]:
def punctuation_removal(sentence):
    # Remove the all the punctuations except '
    new_sentence = re.sub(',|!|\?|\"|<|>|\(|\)|\[|\]|\{|\}|@|#|\+|\=|\-|\_|~|\&|\*|\^|%|\||\$|/|`|\.|\'',
                          '', sentence,count=0, flags=0)
    return new_sentence

## 2.4 Sentence tokenization

In [16]:
def tokenization(sentence):
    new_sentence = nltk.word_tokenize(sentence)
    return new_sentence

## 2.5 Stopwords removal

In [17]:
def stopword_removal(sentence):
    #stoplist = stopwords.words('english')
     
    with open('./stopwords.txt') as file:
        stoplist = [stopword.replace('\n', '').lower() for stopword in file.readlines()]
    
    new_sentence = [word for word in sentence if word not in stoplist]
    return new_sentence

## 2.6 Lemmatization

In [18]:
def get_wordnet_pos(word):
    pack = nltk.pos_tag([word])
    tag = pack[0][1]
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV

    else:
        return None


def lemmatization(sentence):
    lemmatizer = nltk.stem.WordNetLemmatizer()

    new_sentence = [lemmatizer.lemmatize(word, get_wordnet_pos(word) or wordnet.NOUN) for word in sentence]

    return new_sentence

## 2.7 Spelling correction (Optional)

In [20]:
# Package may be used in this section
import nltk
from nltk.corpus import words
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams

# Spell correction is also important in text preprocessing.
# Please refer to the Day2 slides and see how jaccard_distance works
def spell_correction(sentence):
    new_sentence = sentence
    ##############################
    # Your code here
    correct_words = words.words()
    for index, w in enumerate(sentence):
        correction_list = {}
        for word in correct_words:
            if(w[0] == word[0] and len(w) > 2 and len(word) > 2):
                jd = jaccard_distance(set(ngrams(w, 3)), set(ngrams(word, 3)))
                if(jd < 0.15):
                    correction_list[word] = jd
            else:
                correction_list[word] = 0.0
        if(len(correction_list) > 1):
            correction_list = {k: v for k, v in sorted(correction_list.items(), key = lambda item: item[1])}
        for correction in correction_list:
            if(correction == w):
                new_sentence[index] = w
                break
            else:
                new_sentence[index] = list(correction_list)[0]
    ############################## 
    return new_sentence

# example
s = ['I', 'met', 'my', 'boy', 'frienss', 'yesterdsy']
print(spell_correction(s))

KeyboardInterrupt: 

In [47]:
from nltk.util import ngrams

jaccard_distance(set(ngrams("frien", 1)), set(ngrams("friend", 1)))
#print(set(ngrams("friens", 4)))
#print(set(ngrams("friend", 4)))

0.16666666666666666

## 2.8. Integrate all the functions

In [13]:
def text_preprocessing(raw_sentence):
    sentence = lower_casing(raw_sentence)
    sentence = expand_abbriviation(sentence)
    sentence = punctuation_removal(sentence)
    sentence = tokenization(sentence)
    sentence = stopword_removal(sentence)
    sentence = lemmatization(sentence)
    sentence = spell_correction(sentence) # Spelling check
    return sentence


# 3. Lets have a try

In [14]:
# A Common text pre-processing procedure is as follow:
# Raw text -> Lower casing -> Expand abbr -> Punctuation removal->...
# ... -> Tokenization  -> Stop word removal -> Lemmatization
# All the functions are defined in above code blocks. Please feel free to go through it
# and change some of the codes.

# Let's assume we have a raw sentence.
raw_sentence = 'He said, "we\'d have eaten more than 100 hamburgers from yesterdsy."'

# Can you guess the result of procedure?
sentence = text_preprocessing(raw_sentence)
print(sentence)

['eat', '100', 'hamburger', 'yesterday']


In [15]:
# Or if we only want to use some of the preprocessing techniques, we can call the function separately.
sentence = lower_casing(raw_sentence)
sentence = punctuation_removal(sentence)
sentence = expand_abbriviation(sentence)
sentence = tokenization(sentence)
sentence = stopword_removal(sentence)
sentence = lemmatization(sentence)
sentence = spell_correction(sentence) # Spelling check
print(sentence)

['eat', '100', 'hamburger', 'yesterday']


In [16]:
# In the workshop folder, we have a database called questionbase_raw.txt. Try to import the txt file
# into the Python and do the preprocessing for all the sentence. See what will happen!

with open('./questionbase_raw.txt') as file:
    raw_sentences = [sentence.replace('\n', '') for sentence in file.readlines()]

i = 1
for raw_sentence in raw_sentences:
    processed_sentence = text_preprocessing(raw_sentence)
    if raw_sentence != 'Q' and raw_sentence != 'A':
        print(i, raw_sentence)
        print(i, processed_sentence)
        i += 1
        print()

1 Hello
1 []

2 Hello, I am ASD knowledge bot. Feel free to ask me anything about autism spectrum disorder (ASD).
2 ['asd', 'knowledge', 'bot', 'feel', 'free', 'autism', 'spectrum', 'disorder', 'asd']

3 What is definition of Autistic Spectrum Disorder?
3 ['definition', 'autistic', 'spectrum', 'disorder']

4 Autism, or autism spectrum disorder (ASD), refers to a broad range of conditions characterized by challenges with social skills, repetitive behaviors, speech and nonverbal communication. According to the Centers for Disease Control, autism affects an estimated 1 in 54 children in the United States today.
4 ['autism', 'autism', 'spectrum', 'disorder', 'asd', 'refers', 'broad', 'range', 'condition', 'characterize', 'challenge', 'social', 'skill', 'repetitive', 'behavior', 'speech', 'nonverbal', 'communication', 'center', 'disease', 'control', 'autism', 'estimate', '1', '54', 'child', 'united']

5 What are the symptoms of Autistic Spectrum Disorder?
5 ['symptom', 'autistic', 'spectrum

# 3. Text Preprocessing [Workshop]
### Spacy (Python package) is modern and powerful NLP tool. 
### You can use spacy functions do most of the preprocessing work.

In [17]:
# Create the nlp tool
nlp = spacy.load('en_core_web_sm')

# Raw sentence
raw_sentence = 'He said, "we\'d have eaten more than 100 hamburgers from yesterdsy."'

In [18]:
# Use SpaCy nlp tool to process the raw sentence
token_sentence = nlp(raw_sentence)
for token in token_sentence:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop)

He he PRON PRP nsubj Xx True True
said say VERB VBD ROOT xxxx True False
, , PUNCT , punct , False False
" " PUNCT `` punct " False False
we we PRON PRP nsubj xx True True
'd would AUX MD aux 'x False True
have have AUX VB aux xxxx True True
eaten eat VERB VBN ccomp xxxx True False
more more ADJ JJR amod xxxx True True
than than ADP IN quantmod xxxx True True
100 100 NUM CD nummod ddd False False
hamburgers hamburger NOUN NNS dobj xxxx True False
from from ADP IN prep xxxx True True
yesterdsy yesterdsy PROPN NNP pobj xxxx True False
. . PUNCT . punct . False False
" " PUNCT '' punct " False False


In [19]:
token_sentence[0].text

'He'

In [20]:
token_sentence[0].pos_

'PRON'

### Your hands-on time now:

In [21]:
# Used the given information to implement your own text preprocessing
def my_preprocessing(raw_sentence, nlp_tool):
    token_sentence = nlp_tool(raw_sentence) # nlp model return tokenization words
    preprocessed_sentence = None
    
    #########################
    # Your code here
    # You should ignore the abbreviation expanding part here since those abbr words
    # are identified as stop words in spacy.
    #
    # But also consider how to process abbr like U.K. and US such words. 
    # (hint: word2vec, not mentioned in this course)
    
    processing_sentence = []
    for token in token_sentence:
        if(token.is_stop == False and token.pos_ != "PUNCT"): # Stop Word Removal and Removal of Punctuation
            processing_sentence.append(token.lemma_) # return Lemmatization
            
    #########################
    preprocessed_sentence = processing_sentence

    return preprocessed_sentence

In [22]:
# Similarly, call my_preprocessing to process all the text data and
# see what's the difference between the two functions
with open('./questionbase_raw.txt') as file:
    raw_sentences = [sentence.replace('\n', '') for sentence in file.readlines()]
    preprocessed_sentences = [] # result from 'my_preprocessing'
    given_preprocessed_sentences = [] # result by running the 'text_preprocessing'
    
    #########################
    # Your code here
    for raw_sentence in raw_sentences:
        if (raw_sentence == 'Q' or raw_sentence == 'A'):
            continue
        else:
            preprocessed_sentences.append(my_preprocessing(raw_sentence, spacy.load('en_core_web_sm')))
            given_preprocessed_sentences.append(text_preprocessing(raw_sentence))
    #########################
    
    i = 1
    for raw_sentence in raw_sentences:
        if raw_sentence == 'Q' or raw_sentence == 'A':
            continue
        else:
            print(i, raw_sentence)
            print(i, '=>> text prep output:\n', preprocessed_sentences[i-1])
            print(i, '=>> my spacy  output:\n', given_preprocessed_sentences[i-1])
            i += 1
            print()

1 Hello
1 =>> text prep output:
 ['hello']
1 =>> my spacy  output:
 []

2 Hello, I am ASD knowledge bot. Feel free to ask me anything about autism spectrum disorder (ASD).
2 =>> text prep output:
 ['hello', 'asd', 'knowledge', 'bot', 'feel', 'free', 'ask', 'autism', 'spectrum', 'disorder', 'ASD']
2 =>> my spacy  output:
 ['asd', 'knowledge', 'bot', 'feel', 'free', 'autism', 'spectrum', 'disorder', 'asd']

3 What is definition of Autistic Spectrum Disorder?
3 =>> text prep output:
 ['definition', 'Autistic', 'Spectrum', 'Disorder']
3 =>> my spacy  output:
 ['definition', 'autistic', 'spectrum', 'disorder']

4 Autism, or autism spectrum disorder (ASD), refers to a broad range of conditions characterized by challenges with social skills, repetitive behaviors, speech and nonverbal communication. According to the Centers for Disease Control, autism affects an estimated 1 in 54 children in the United States today.
4 =>> text prep output:
 ['autism', 'autism', 'spectrum', 'disorder', 'ASD', '

In [23]:
# Compare the two results and explain which one is better and why?
# Provide your answer here
# Chang Siang's Answer:
# It seems like the model using Spacy's library performs better than the text_processing model that was constructed by hand
# Spacy NLP model were able to capture the context of certain words like "United States", while the text_preprocessing dropped the word "state", resulting on loss of meaning.
# Furthermore, certain words like "get", "have", "make", "able" which provides context to the sentence meaning were preserve when using spacy model, and was dropped as "stop word" in text_preprocessing model.
# Taken together, my judgement is Spacy is a better library to use for nlp preprocessing over nltk library.

---
`The end is called a new start.` --- ISS: **I** **S**(elf) **S**(tudy)

---